# Groupby and Aggregate Functions on Spark Dataframe

## Table of Content

<ol style = "type:1">
    <li><a href = "#groupbyagg"><code>`groupBy`</code> and <code>`agg`</code></a></li>
    <li><a href = "#importfuncfromspark">Importing Functions from Spark</a></li>
    <li><a href = "#sorting">Sorting</a></li>
    <li><a href = "#ref">References</a></li>
</ol>

## <a name = "groupbyagg">`groupBy` and `agg`</a>

In [1]:
from pyspark.sql import SparkSession

In [ ]:
# Output hidden
spark = SparkSession.builder.appName("aggs").getOrCreate()

In [3]:
df = spark.read.csv("sales_info.csv", 
                    inferSchema = True, 
                    header = True
                   )

Let's see what the data looks like.

In [4]:
df.show(5)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
+-------+-------+-----+
only showing top 5 rows



In [5]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



To group together by the `Company` column, we call the `.groupBy()` method. `df.groupBy(column_name)` is a `group.GroupedData` object; and Python shows you its memory location.

In [6]:
df.groupBy("Company")

A vareity of methods can be called on this `GroupedData` object, such as:

In [7]:
df.groupBy("Company").mean()

DataFrame[Company: string, avg(Sales): double]

In [8]:
df.groupBy("Company").mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [9]:
df.groupBy("Company").max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [10]:
df.groupBy("Company").sum().show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



In [11]:
df.groupBy("Company").count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



etc. Now, not all methods need a `.groupBy()`-call. A generalised `agg` method can be called to aggregate across all rows in the data.

In [13]:
df.agg({"Sales": "sum"}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



This returns the sum of all the sales in the dataframe. We can also have

In [14]:
df.agg({"Sales": "max"}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



This dictionary syntax also works on `groupBy` as well.

In [15]:
group_data = df.groupBy("Company")

In [16]:
group_data.agg({"Sales": "max"}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



## <a name = "importfuncfromspark">Importing Functions from Spark</a>

In [18]:
from pyspark.sql.functions import countDistinct, avg, stddev

We can combine these functions with a `.select()` call.

In [20]:
df.select(avg("Sales")).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



We can change the title `avg(Sales)` with alias. Pay attention that `.alias()` and the function should be inside the same parentheses.

In [21]:
df.select(avg("Sales").alias("Average Sales")).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



Let us select the standard deviation.

In [22]:
df.select(stddev("Sales")).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



This is a lengthy number, and we can make it look nicer.

In [23]:
from pyspark.sql.functions import format_number

In [27]:
sales_std = df.select(stddev("Sales").alias("std"))

In [28]:
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [30]:
sales_std.select(format_number("std", 2).alias("std")).show()

+------+
|   std|
+------+
|250.09|
+------+



## <a href = "sorting">Sorting</a>

In [32]:
df.orderBy("Sales").show(10)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
+-------+-------+-----+
only showing top 10 rows



Decending is tricky, since we need to pass in the actual column than the column's name, and call `.desc()`.

In [33]:
df.orderBy(df["Sales"].desc()).show(10)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
+-------+-------+-----+
only showing top 10 rows



## <a name = "ref">References</a>

<ol style = "type:1">
    <li>Jose Portilla. Spark and Python for Big Data with PySpark.</li>
    <li>Apache Spark. <a href = "https://spark.apache.org/docs/latest/api/python/">https://spark.apache.org/docs/latest/api/python/</a>.</li>
</ol>